In [1]:
### 데이터셋 구조
# container_dataset
# ├── train_images
# │   ├── OBJ00013_PS3_K3_NIA0078.png
# ...................................
# ├── train_lbl
# │   ├── OBJ00013_PS3_K3_NIA0078.txt
# ...................................
# ├── valid_lbl
# │   ├── OBJ00023_PS3_K3_NIA0010.txt
# train_lbl과 valid_합쳐서 train_images 전체로 구성되도록 해야함
# 이미지가 아주 많지 않기 때문에 8:2로 나누어서 먼저 진행해보기
# 원본 라벨 데이터인 train_labels는 그대로 놔두기 
# valid_images가 실제 실험데이터임

In [7]:
import json
import glob as _glob
import os
import random
import cv2

In [8]:
def glob(dir, pats, recursive=False):  # faster than match, python3 only
    pats = pats if isinstance(pats, (list, tuple)) else [pats]
    matches = []
    for pat in pats:
        matches += _glob.glob(os.path.join(dir, pat), recursive=recursive)
    return matches
def csv2list(filename):
  lists=[]
  file=open(filename,"r")
  while True:
    line=file.readline().replace('\n','')
    if line:
      line=line.split(",")
      lists.append(line)
    else:
      break
  return lists

In [4]:
all_lbls = glob('container_dataset/train_labels','*')
random.shuffle(all_lbls)

In [5]:
print(all_lbls)

['container_dataset/train_labels/OBJ00013_PS3_K3_NIA0078.json', 'container_dataset/train_labels/OBJ03780_PS3_K3A_NIA0158.json', 'container_dataset/train_labels/OBJ03422_PS3_K3A_NIA0139.json', 'container_dataset/train_labels/OBJ03730_PS3_K3A_NIA0156.json', 'container_dataset/train_labels/OBJ03652_PS3_K3A_NIA0153.json', 'container_dataset/train_labels/OBJ04768_PS3_K3A_NIA0314.json', 'container_dataset/train_labels/OBJ03413_PS3_K3A_NIA0138.json', 'container_dataset/train_labels/OBJ05720_PS3_K3A_NIA0363.json', 'container_dataset/train_labels/OBJ04640_PS3_K3A_NIA0305.json', 'container_dataset/train_labels/OBJ03964_PS3_K3A_NIA0166.json', 'container_dataset/train_labels/OBJ04440_PS3_K3A_NIA0291.json', 'container_dataset/train_labels/OBJ05677_PS3_K3A_NIA0360.json', 'container_dataset/train_labels/OBJ05440_PS3_K3A_NIA0165.json', 'container_dataset/train_labels/OBJ05205_PS3_K3A_NIA0340.json', 'container_dataset/train_labels/OBJ04493_PS3_K3A_NIA0296.json', 'container_dataset/train_labels/OBJ05124

In [6]:
train_lbls = all_lbls[:int(len(all_lbls)*0.8)]

valid_lbls = all_lbls[int(len(all_lbls)*0.8):]

In [7]:
print(len(train_lbls),len(valid_lbls))

217 55


In [8]:
for lbl in train_lbls:
    with open(lbl, 'r') as file:
        data = json.load(file)
    for data_row in data['features']:
        INdata = list(map(str,map(int,(map(float, data_row['properties']['object_imcoords'].split(','))))))
        INdata = ' '.join(INdata)
        INdata +=' container 0'

        with open("container_dataset/train_lbl/" + lbl.replace('\\', '/').split('/')[-1].replace('.json', '.txt'), "a") as f:
            f.write(INdata+'\n')

In [9]:
for lbl in valid_lbls:
    with open(lbl, 'r') as file:
        data = json.load(file)
    for data_row in data['features']:
        INdata = list(map(str,map(int,(map(float, data_row['properties']['object_imcoords'].split(','))))))
        INdata = ' '.join(INdata)
        INdata +=' container 0'

        with open("container_dataset/valid_lbl/" + lbl.replace('\\', '/').split('/')[-1].replace('.json', '.txt'), "a") as f:
            f.write(INdata+'\n')

In [9]:
### 홍진성 추가 
### VALIDATION SET을 RANDOM CROP과 유사하게 SET하기 위해서 5배 확대이후에 1024,1024로 잘라서 VALIDSET 새로 만듬
### 이미지의 가장자리에 위치해서 잘리는경우에는 버림
valid_labels = glob('container_dataset/valid_lbl','*')


for val_lbl in valid_labels:
    rboxls = csv2list((val_lbl))
    img = cv2.imread('container_dataset/train_images/' + val_lbl.replace('\\', '/').split('/')[-1][:-3] + 'png')
    img = cv2.resize(img,(5120,5120))
    for i in range(5):
        for j in range(5):
            WS = j * 1024
            WE = (j + 1) * 1024
            HS = i * 1024
            HE = (i + 1) * 1024

            patch = img[HS:HE, WS:WE, :]

            for lbl in rboxls:
                pos_8 = list(map(int, lbl[0].split()[:8]))
                if WS <= pos_8[0]*5 < WE and HS <= pos_8[1]*5 < HE and WS <= pos_8[2]*5 < WE and HS <= pos_8[3]*5 < HE \
                        and WS <= pos_8[4]*5 < WE and HS <= pos_8[5]*5 < HE and WS <= pos_8[6]*5 < WE and HS <= pos_8[7] * 5 < HE:
                    pos_8[0] = pos_8[0] * 5 - WS
                    pos_8[1] = pos_8[1] * 5 - HS
                    pos_8[2] = pos_8[2] * 5 - WS
                    pos_8[3] = pos_8[3] * 5 - HS
                    pos_8[4] = pos_8[4] * 5 - WS
                    pos_8[5] = pos_8[5] * 5 - HS
                    pos_8[6] = pos_8[6] * 5 - WS
                    pos_8[7] = pos_8[7] * 5 - HS

                    pos_8 = list(map(str, pos_8))
                    INdata = ' '.join(pos_8)
                    INdata += ' container 0'

                    with open("container_dataset/train_VAL_patch_lbl/" + val_lbl.replace('\\', '/').split('/')[-1][:-4] + str(j) + str(i)+'.txt',"a") as f:
                        f.write(INdata + '\n')

            cv2.imwrite('container_dataset/train_VAL_patch_img/'+val_lbl.replace('\\', '/').split('/')[-1][:-4] + str(j) + str(i)+'.png', patch)